# Plotting the OGGM surface mass-balance, the ELA and AAR

A few recipes to help you to plot and analyse the mass-balance computed by OGGM.

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import numpy as np
import pandas as pd

import oggm
from oggm import cfg, utils, workflow, tasks, graphics, global_tasks
from oggm.core import massbalance, flowline

In [ ]:
cfg.initialize(logging_level='WARNING')
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='OGGM-ref-mb', reset=True)
cfg.PARAMS['border'] = 80
cfg.PARAMS['store_model_geometry'] = True

We use Hintereisferner:

In [ ]:
# (The second glacier here we will only use in the second part of this notebook.)
gdirs = workflow.init_glacier_directories(['RGI60-11.00897', 'RGI60-11.00001'], from_prepro_level=3)
gdir = gdirs[0]
tasks.init_present_time_glacier(gdir)

## With static geometry

That's by far the easiest - that is how we mostly test and develop mass-balance models ourselves.

### The very easy way 

In [ ]:
# Get the calibrated mass-balance model - the default is to use OGGM's "PastMassBalanceModel"
mbmod = massbalance.MultipleFlowlineMassBalance(gdir)

# Compute the specific MB for this glacier
fls = gdir.read_pickle('model_flowlines')
years = np.arange(1950, 2019)
mb_ts = mbmod.get_specific_mb(fls=fls, year=years)

plt.plot(years, mb_ts);
plt.ylabel('Specific MB (mm w.e.)');
plt.xlabel('Year');

In this particular case (Hintereisferner), we can also plot the measured data:

In [ ]:
# Get the reference mass-balance from the WGMS
ref_df = gdir.get_ref_mb_data()
# Get OGGM MB at the same dates
ref_df['OGGM'] = mbmod.get_specific_mb(fls=fls, year=ref_df.index.values)
# Plot
plt.plot(ref_df.index, ref_df.OGGM, label='OGGM');
plt.plot(ref_df.index, ref_df.ANNUAL_BALANCE, label='WGMS');
plt.ylabel('Specific MB (mm w.e.)'); plt.legend();
plt.xlabel('Year');

### Mass-balance profiles on flowlines

Let's start by noting that I used the `MultipleFlowlineMassBalance` model here. This is what OGGM uses for its runs, because we allow different model parameters for different flowlines. In many cases we don't need this, but sometimes we do. Let's see if we have different temperature sensititivites for Hintereisferner or not:

In [ ]:
for flmod in mbmod.flowline_mb_models:
    print(flmod.mu_star)

So here we have the same model parameters for each flowline (that's frequently the case). To get the mass-balance as a function of height we have several possibilities:

In [ ]:
# Where the flowlines are:
heights, widths, mb = mbmod.get_annual_mb_on_flowlines(fls, year=2000)
# units kg ice per second to mm w.e. per year:
mb = mb * cfg.SEC_IN_YEAR * cfg.PARAMS['ice_density'] 

# Plot
plt.plot(mb, heights, '*', label='2000');

# Another year:
heights, widths, mb = mbmod.get_annual_mb_on_flowlines(fls, year=2001)
# units kg ice per second to mm w.e. per year:
mb = mb * cfg.SEC_IN_YEAR * cfg.PARAMS['ice_density'] 
# Plot
plt.plot(mb, heights, '*', label='2001');
plt.ylabel('Elevation (m a.s.l.)'); plt.xlabel('MB (mm w.e. yr$^{-1}$)'); plt.legend();

### Mass-balance at any height

For this, we can either "pick" one of the single flowline mass-balance models or use the multiple one (it's the same):

In [ ]:
heights = np.arange(2500, 4000)
# Method 1
mbmod1 = mbmod.flowline_mb_models[0]
mb = mbmod1.get_annual_mb(heights, year=2001) * cfg.SEC_IN_YEAR * cfg.PARAMS['ice_density'] 
# Method 2
mb2 = mbmod.get_annual_mb(heights, year=2001, fl_id=0) * cfg.SEC_IN_YEAR * cfg.PARAMS['ice_density'] 
np.testing.assert_allclose(mb, mb2)
# Plot
plt.plot(mb, heights, label='2001');
plt.ylabel('Elevation (m a.s.l.)'); plt.xlabel('MB (mm w.e. yr$^{-1}$)'); plt.legend();

But again, for certain glaciers the mass-balance might change from one flowline to another!

## Mass-balance during a transient run + bonus info (timestamps in OGGM)

### Specific (glacier-wide) MB 

At the time of writing, we only store the glacier volume, which is roughly equivalent to the mass-balance (up to numerical accuracy in the dynamical model).

In [ ]:
# Run from outline inventory year (2003 for HEF) to 2017 (end of CRU data in hydro year convention, see below)
tasks.run_from_climate_data(gdir, ye=2020);

In [ ]:
ds_diag = xr.open_dataset(gdir.get_filepath('model_diagnostics'))
ds_diag.volume_m3.plot(); plt.title('Volume evolution of Hintereisferner');

### Important: timestamps in OGGM

This can be a bit of a headache, so let's spend some time explaining this. OGGM was developed to follow the traditional "hydrological year" convention. So in the OGGM output files (and in the model internals), we use the same convention (history will tell if it was a good idea, I sometimes [regret it](https://github.com/OGGM/oggm/issues/1020)).

We use what we call a "floating year" time stamp, which can be converted to real dates easily in the model:

In [ ]:
from oggm.utils import floatyear_to_date, hydrodate_to_calendardate
print('Hydro dates')
print(floatyear_to_date(2003.0))
print(floatyear_to_date(2003.99))
print('Calendar dates')
print(hydrodate_to_calendardate(*floatyear_to_date(2003.0), start_month=10))
print(hydrodate_to_calendardate(*floatyear_to_date(2003.99), start_month=10))

The timestamp used in the OGGM output files uses the same convention:

In [ ]:
print(ds_diag.time.values[0], ds_diag.calendar_year.values[0], ds_diag.calendar_month.values[0])

So the simulation started in year 2004.0 (as of the OGGM update from 19.05.2020), which corresponds to the real date October 2003. This is in accordance with the RGI inventory date (probably based on satellite imagery taken in the summer 2003):

In [ ]:
gdir.rgi_date

Similarly, the simulation ends at the last possible date for which we have climate data for (i.e. the simulation stops at the end of September 2019, which translates to the beginning of hydrological year 2020):

In [ ]:
print(ds_diag.time.values[-1], ds_diag.calendar_year.values[-1], ds_diag.calendar_month.values[-1])

Therefore, you have to be careful about how you interpret these data or report them to others.

### Specific MB from volume change time series 

We compute the specific MB from the delta volume divided by the glacier area, and compare it to observations and to the mass-balance computed with a fixed geometry:

In [ ]:
# Specific MB over time (i.e. with changing geometry feedbacks)
smb = (ds_diag.volume_m3.values[1:] - ds_diag.volume_m3.values[:-1]) / ds_diag.area_m2.values[1:]
smb = smb * cfg.PARAMS['ice_density']  # in mm
plt.plot(ds_diag.time[:-1], smb, label='OGGM (dynamics)'); 
# The SMB from WGMS and fixed geometry we already have
plt.plot(ref_df.loc[2004:].index, ref_df.loc[2004:].OGGM, label='OGGM (fixed geom)');
plt.plot(ref_df.loc[2004:].index, ref_df.loc[2004:].ANNUAL_BALANCE, label='WGMS');
plt.legend();

A few comments:
- the systematic difference between WGMS and OGGM is due to the calibration of the model, which is done with fixed geometry (the bias is zero over the entire calibration period but changes sign over the 50 year period, see plot above)
- the difference between OGGM dynamics and fixed geometry (so small that they are barely visible for this short time period) is due to:
    - the changes in geometry during the simulation time (i.e. this difference grows with time)
    - melt at the tongue which might be larger than the glacier thickness is not accounted for in the fixed geometry data (these are assumed to be small)
    - numerical differences in the dynamical model (these are assumed to be small as well)

### MB profiles

At time of writing, we do not store the mass-balance profiles during a transient run, although we could (and [should](https://github.com/OGGM/oggm/issues/1022)). To get the profiles with a dynamically changing geometry, there is no other way than either get the mass-balance during the simulation, or to fetch the model output and re-compute the mass-balance retroactively. 

**Note: the MB profiles themselves of course are not really affected by the changing geometry - what changes from one year to another is the altitude at which the model will compute the MB.** In other terms, if you are interested in the mass-balance profiles you are better of to use the fixed geometries approaches explained at the beginning of the notebook.

#### Retrospective MB from model output 

In [ ]:
# This reads the model output file and makes it look like an OGGM flowline model
dyn_mod = flowline.FileModel(gdir.get_filepath('model_geometry'))
# Get the calibrated mass-balance model
mbmod = massbalance.MultipleFlowlineMassBalance(gdir)
# We can loop over the years and read the flowlines for each year
for year in range(2004, 2017):
    dyn_mod.run_until(year)
    # Compute the SMB from the mass balance model and the flowlines heights at each year
    h = []
    smb = []
    for fl_id, fl in enumerate(dyn_mod.fls):
        h = np.append(h, fl.surface_h)
        mb = mbmod.get_annual_mb(fl.surface_h, year=year, fl_id=fl_id) 
        smb = np.append(smb, mb * cfg.SEC_IN_YEAR * cfg.PARAMS['ice_density'] )
    plt.plot(smb, h, '.', label=year)
plt.legend();

#### Compute the MB during the simulation 

In [ ]:
# TODO - also based on the new outputs

## Equilibrium Line Altitude 

The second part of this notebook shows how you can compute the Equilbrium Line Altitude (ELA, the altitude at which the MB is equal to 0) with OGGM.

As the ELA in OGGM only depends on the [mass balance model](https://docs.oggm.org/en/stable/mass-balance.html) itself (not on glacier geometry), there is no need to do a full model run to collect these values. This is also the reason why the ELA is no longer a part of the model run output, but is instead  a diagnostic variable that can be computed separately since OGGM v1.6.

### Compute and compile the ELA 

In [ ]:
global_tasks.compile_ela(gdirs, ys=2000, ye=2019);

Per default, the file will be stored in the working directory and named `ELA.hdf` (you can choose to save the data in a `csv` file instead by setting the `csv` keyword to `True`). The location where the file is saved can be changed by specifying the `path` keyword.

Let's read the file and plot it:

In [ ]:
# Read the file
ela_df = pd.read_hdf(os.path.join(cfg.PATHS['working_dir'], 'ELA.hdf'))

# Plot it
ela_df.plot();
plt.xlabel('year'); plt.ylabel('ELA [m]');

When looking at many glaciers it might be useful to also plot the mean or average of the data. Let's plot the mean ELA and the glacier area weighted ELA average:

In [ ]:
areas = [gd.rgi_area_km2 for gd in gdirs]

avg = ela_df.mean(axis=1).to_frame(name='mean')
avg['weighted average'] = np.average(ela_df, axis=1, weights=areas)

avg.plot();
plt.xlabel('year'); plt.ylabel('ELA [m]');

How would to ELA look like if the climate is 1° warmer? Lets have a look at one of the glaciers:

In [ ]:
global_tasks.compile_ela(gdirs, ys=2000, ye=2019, temperature_bias=1, filesuffix='_t1')
ela_df_t1 = pd.read_hdf(os.path.join(cfg.PATHS['working_dir'], 'ELA_t1.hdf'))

In [ ]:
plt.plot(ela_df.mean(axis=1), label='default climate')
plt.plot(ela_df_t1.mean(axis=1), label=('default climate +1℃'))
plt.xlabel('year CE'); plt.ylabel('ELA [m]'); plt.legend();

By using the `precipitation_factor` keyword, you can change the precipitation. Feel free to try that by yourself!

Lets look at a longer timeseries, for one glacier only: 

In [ ]:
global_tasks.compile_ela(gdirs[0], ys=1902, ye=2019, filesuffix='_1901_2019', csv=True)
ela_df_long = pd.read_csv(os.path.join(cfg.PATHS['working_dir'], 'ELA_1901_2019.csv'), index_col=0)

The ELA can have a high year-to-year variability. Therefore we plot in addition to the regular ELA timeseries, the 5-year moving average. 

In [ ]:
ax = ela_df_long.plot();
ela_df_long.rolling(5).mean().plot(ax=ax, lw=2, color='k', label='')
plt.xlabel('year CE'); plt.ylabel('ELA [m]'); ax.legend(["Annual", "5-yr average"]);

In case you're only intrested in specific years there is an option to just compute the ELA in those years. There is actually no need to save this data. Therefore we now just compute the ELA. 

In [ ]:
yrs = [2000, 1950, 1980, 1960]
ela_yrs = massbalance.compute_ela(gdirs[0], years=yrs)
ela_yrs

See in this case you can use any year as long as you have climate available for that year. However for plotting purposes it might be worth to sort the data, otherwise the following happens ;)

In [ ]:
ela_yrs.plot();

In [ ]:
ela_yrs.sort_index(ascending=True).plot();
plt.xlabel('year CE'); plt.ylabel('ELA [m]');

By now we have addressed most of the keyword agruments of the compile_ela and compute_ela functions. To use different climate than the default climate, you can make use of the `climate_filename` and `climate_input_filesuffix` keywords. In case you're not familiar with those yet, please check out the [run_with_gcm](run_with_gcm.ipynb) notebook.

# Accumulation Area Ratio 

Like the ELA, the Accumulation Area Ratio (AAR) is a diagostic variable. The AAR is currently not an output 
variable in OGGM, but it can easily be computed. It is even a part of the [glacier simulator](https://bokeh.oggm.org/simulator/app) and its [documentation](https://edu.oggm.org/en/latest/simulator.html#aar-accumulation-area-ratio). Below we give an example of how it can be computed after a model run. From the ELA 
computation, we already know the height where the mass balance is equal to zero. Now we need to compute the area 
of the glacier that is located above that line. Lets start with a static glacier with multiple flowlines.

We first search the points along the flowlines that are above the ELA in a given year. Lets start with ELA in 2010
and use again Hintereisferner. Here we check if the ELA lays below the flowline and if that is not the case which 
point along the flowline is the first below the ELA.

In [ ]:
ELA_2010 = ela_df['RGI60-11.00897'][2010] 

p_ELA = np.empty(len(fls)) 
for fn in np.arange(len(fls)):
    MB_check = fls[fn].surface_h < ELA_2010 
    if MB_check[-1] == False:
        p_ELA[fn] = len(fls[fn].surface_h)
    else:
        p_ELA[fn] = np.where(MB_check)[0][0] 
p_ELA 

Using these points we can make a quick estimate of the AAR, by adding up the area above these points allong the 
flowline and diving by the total glacier area. 

In [ ]:
AC = np.empty(len(fls))

for fn in np.arange(len(fls)):
    if p_ELA[fn] == 0:
        AC[fn] = 0
    else:
        AC[fn] = fls[fn].bin_area_m2[0:int(p_ELA[fn])].sum()
    
AAR_estimate = AC.sum()/gdir.rgi_area_m2
AAR_estimate

Now we have looked at the static case we can continue with looking at dynamic glaciers.

In [ ]:
accumulation_area = np.empty(len(fls)) # initialize the arrays we need later on.
total_area = np.empty(len(fls))
AAR = np.empty(10)

for kt in np.arange(2005,2015): # loop over the years of intrest
    for fn in np.arange(len(fls)): # loop over the flowlines
        geometry = xr.open_dataset(gdir.get_filepath('model_geometry'), group='fl_' + str(fn))
        surface_h = (geometry.loc[dict(time=kt)].ts_section[:] / geometry.loc[dict(time=kt)].ts_width_m[:]) \
                                    + geometry.bed_h # calculate the surface heigth of the glacier
        # select points the points in the accumulation zone and compute their area.
        accumulation_area[fn] = np.sum(np.where(surface_h > ela_df['RGI60-11.00897'][kt],
                                    geometry.loc[dict(time=kt)].ts_width_m[:], 0) 
                                    * geometry.dx * geometry.map_dx)
        # compute the total area along the flowline
        total_area[fn] = np.sum(geometry.loc[dict(time=kt)].ts_width_m[:] * geometry.dx * geometry.map_dx)
    # compute the AAR in a given year
    AAR[int(kt-2005)] = (np.sum(accumulation_area)/np.sum(total_area))

Keep in mind that this is just an example for computing the AAR. For a different glacier experiment you might need
to adjust the code. e.g. if the glacier flowlines would have different temperature sensitivities, you need to take
that in to consideration.  

Here we plot the results. 

In [ ]:
plt.plot(np.arange(2005,2015), AAR, label='AAR from dynamic run')
plt.scatter(2010, AAR_estimate, label='AAR example from static glacier')
plt.xlim([2005,2014])
plt.ylim([0,1])
plt.xlabel('year CE')
plt.ylabel('AAR')
plt.legend()
plt.tight_layout()

You can see in the plot that the difference between the AAR calculated from the dynamic glacier and the one of the 
static glacier is very similar. Over time you can expect that this difference becomes larger. Feel free to play 
around with that. 

Here we computed the AAR from the "model perspective", where the glacier is located in bins. You can however also 
choose to do some interpolation between the grid point above and below the ELA. 

## What's next?

- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)